In [1]:
import plotly.express as px
import numpy as np
from tqdm import tqdm
import pandas as pd

from scipy.spatial import distance_matrix

from matplotlib import pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os
from MPSE.MPSE import mview
import math
from utils import *
import json

# import pyvista as pv

%matplotlib inline

2022-11-01 10:31:08.880205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 10:31:08.955306: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-01 10:31:09.277509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/insane/.cache/pypoetry/virtualenvs/3d-reconstruction-GT1uMocM-py3.10/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-01 10:31:09.277549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64]

In [2]:
DATA_DIR = '/run/media/insane/My 4TB 2/Big Data/MPSE/Data/ShapeNetCore/ShapeNetCore.v2'

In [85]:
n = 0

In [100]:
n -= 2

In [279]:
# Figure 3 in LMNet paper



model_id = '04379243'
            
for i, obj_id in enumerate(os.listdir(os.path.join(DATA_DIR, model_id))):
    if i < n:
        continue
    n += 1
    obj_path = os.path.join(DATA_DIR, model_id, obj_id, 'models/model_normalized.obj')
    print(i, '/', len(os.listdir(os.path.join(DATA_DIR, model_id))), model_id, obj_id)
    mesh = trimesh.load(obj_path)
    # plot_3D_paper([points], colors=['fancy']).show()
    break

    
angle = math.pi / 4
direction = [1, 1, 0]
center = [0, 0, 0]
rot_matrix = trimesh.transformations.rotation_matrix(angle, direction, center)
mesh.apply_transform(rot_matrix)


# angle = math.pi / 1.8
# direction = [0, 1, 0]
# center = [0, 0, 0]
# rot_matrix = trimesh.transformations.rotation_matrix(angle, direction, center)
# mesh.apply_transform(rot_matrix)


mesh.show()

190 / 8436 04379243 d177f7632a4bb3405073c32a9cf4a6a9


In [280]:
models_to_use = {
    'airplane': [
        '103c9e43cdf6501c62b600da24e0965', # S1-1
        '105f7f51e4140ee4b6b87e72ead132ed', # S1-6
        '10e4331c34d610dacc14f1e6f4f4f49b', # 3-2
        'd405b9e5f942fed5efe5d5ae25ee424e', # S1-1
        '157bb84c08754307dff9b4d1071b12d7', # S1-3
        '8cf06a71987992cf90a51833252023c7' # 3-1
    ],
    'bench': [
        '42ffe8d78c2e8da9d40c07d3c15cc681', # 3-4
        'cad0a0e60708ab662ab293e158725cf0', # S1-8
        'cca18c7f8636606f51f77a6d7299806', # S1-7
        '89e2eaeb437cd42f85e40cb3507a0145', # S1-12
        '702870d836ea3bf32056b4bd5d870b47', # S1-11
        'fc0486ec53630bdbd2b12aa6a0f050b3' # S1-10
    ],
    'car': [
        '44f30f4c65c3142a16abce8cb03e7794', # 3-6
        'd9034b15c7f295551a46c391b387480b', # S1-17
        '35de0d0cc71179dc1a98dff5b6c5dec6', # S1-13
        'd6f8cfdb1659142814fccfc8a25361e', # S1-16
        'd79f66a4566ff981424db5a60837de26' # S1-15
    ],
    'chair': [
        'bf91d0169eae3bfdd810b14a81e12eca', # 3-8
        '6a3d2feff3783804387379bbd607d69e', # 1-7
        'cd6a8020b69455dbb161f36d4e309050', # S1-20 
        'cd9702520ad57689bbc7a6acbd8f058b' # S1-21
    ],
    'lamp': [
        '102273fdf8d1b90041fbc1e2da054acb', # 3-9
        'fa0a32c4326a42fef51f77a6d7299806', # S2-19
        'e6d62a37e187bde599284d844aba7576', # S2-18
    ],
    'rifle': [
        '10cc9af8877d795c93c9577cd4b35faa', # S2-2
        '81ba8d540499dd04834bde3f2f2e7c0c', # S2-4
        '823b97177d57e5dd8e0bef156e045efe', # 3-10
        'f55544d331eb019a1aca20a2bd5ca645', # S2-3
    ],
    'table': [
        '105b9a03ddfaf5c5e7828dbf1991f6a4', # S2-11
        'c3884d2d31ac0ac9593ebeeedbff73b',  # S2-15
        '16961ddf69b6e91ea8ff4f6e9563bff6', # S2-16
        '86e6ef5ae3420e95963080fd7249126d', # S2-14 
    ],
    'sofa': [
        '79bea3f7c72e0aae490ad276cd2af3a4', # S2-8
        'cff485b2c98410135dda488a4bbb1e1', # S2-5
        'd5a2b159a5fbbc4c510e2ce46c1af6e', # S2-6
        'd8c748ced5e5f2cc7e3820d17093b7c2', # 3-5
    ]
}

In [310]:
dataset = f'ShapeNet:sofa:{models_to_use["sofa"][10]}'
mesh = get_shapenet_mesh(dataset, datadir=DATA_DIR)
mesh.show()

In [311]:
points = get_dataset_points(dataset, datadir=DATA_DIR, n_points=4096)

points = np.dot(points, getRotationMatrix(-100, 'x'))

plot_3D_paper([points], point_size=2, colors=['#0c3a6d'])